# Scripts del Proyecto

### Script 1: Preparacion de datos para el entrenamiento

In [2]:
import numpy as np 
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [28]:
df = pd.read_csv("../data/raw/data.csv")

In [29]:
# Eliminar las primeras dos columnas con data innecesaria
df.drop(df.iloc[:, [0,1]], axis=1, inplace=True)

#  Columnas con valores categoricos pueden ser del tipo 'object' o 'int64'
categorical_indexes = [0, 1, 3, 4] + list(range(6,20))
df.iloc[:, categorical_indexes] = df.iloc[:, categorical_indexes].astype('category')

# Como el porcentaje de los calores vacios es menor al 1% podriamos eliminar o reemplaxar por la mediana estos valores
df = df.dropna()

In [30]:
# Dividir la data en numerica y categorica
numerical_columns = [c for c in df.columns if df[c].dtype.name != 'category']
numerical_columns.remove('satisfaction')
categorical_columns = [c for c in df.columns if df[c].dtype.name == 'category']
df_describe = df.describe(include = ['category'])

# Divir las columnas categoricas en binarias y no binarias
binary_columns = [c for c in categorical_columns if df_describe[c]['unique'] == 2]
nonbinary_columns = [c for c in categorical_columns if df_describe[c]['unique'] > 2]

In [31]:
# Codificación
original_df = df[binary_columns].copy()
df[binary_columns] = df[binary_columns].astype('category').apply(lambda x: x.cat.codes)
df[binary_columns] = df[binary_columns].astype('category')

# Tabla de equivalencia
list_transformation = []
list_transformation = [(col, val, code) for col in binary_columns
                                        for val, code in zip(original_df[col].unique(), df[col].unique())]

df_transform = pd.DataFrame(list_transformation, columns=["Nombre columna", "Valor Original", "Categoria"])

In [32]:
df_nonbinary = pd.get_dummies(df[nonbinary_columns])
df_numerical = df[numerical_columns]
df_numerical = (df_numerical - df_numerical.mean(axis = 0))/df_numerical.std(axis = 0)
df_final = pd.concat((df_numerical, df_nonbinary, df[binary_columns], df['satisfaction']), axis = 1)

In [40]:
feature_nonbinary_columns = df_final.columns.to_list()
df_columns = pd.DataFrame(feature_nonbinary_columns, columns=["Columns"])
df_columns.to_csv("../data/raw/columns_train.csv", index=False)

In [41]:
df_final.to_csv("../data/processed/data_train.csv", index=False)

### Script 2: Código de Entrenamiento

In [1]:
import numpy as np 
import pandas as pd
import pickle
import warnings

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("../data/processed/data_train.csv")

In [3]:
y = df['satisfaction']
X = df.drop(['satisfaction'],axis=1)
N, d = X.shape

In [4]:
y

0         neutral or dissatisfied
1         neutral or dissatisfied
2                       satisfied
3         neutral or dissatisfied
4                       satisfied
                   ...           
103589    neutral or dissatisfied
103590                  satisfied
103591    neutral or dissatisfied
103592    neutral or dissatisfied
103593    neutral or dissatisfied
Name: satisfaction, Length: 103594, dtype: object

In [5]:
# Se obtiene el Target y las caracteristicas
features = X.columns.tolist()

In [8]:
# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size = 36000, test_size = 4000,
                                                    stratify = y, random_state = 2023)

In [9]:
y_test.to_csv("../data/processed/target_validate.csv", index=False)
X_test.to_csv("../data/processed/features_validate.csv", index=False)

#### Modelo de Neural Network

In [10]:
alpha_local_opt = 5.623413251903491
hidden_layer_local_opt_1 = 40
hidden_layer_local_opt_2 = 21

In [11]:
# Se corre el modelo con los hiperparametros optimos y se corre con la data para ver los resultados estadisticos del modelo.
mlp_model = MLPClassifier(alpha = alpha_local_opt,
                          hidden_layer_sizes = (hidden_layer_local_opt_1, hidden_layer_local_opt_2),
                          solver = 'lbfgs',
                          max_iter = 1000,
                          activation = 'logistic',
                          random_state = 42)

mlp_model.fit(X_train, y_train)

MLPClassifier(activation='logistic', alpha=5.623413251903491,
              hidden_layer_sizes=(40, 21), max_iter=1000, random_state=42,
              solver='lbfgs')

In [12]:
# Save Best Model
filename = '../models/best_model.pkl'
pickle.dump(mlp_model, open(filename, 'wb'))

### Script 4: Código de Validación

In [13]:
import numpy as np
import pandas as pd
import pickle
import warnings

from sklearn.metrics import roc_auc_score, \
                            accuracy_score, \
                            f1_score, \
                            precision_score, \
                            recall_score, \
                            classification_report, \
                            confusion_matrix

warnings.filterwarnings("ignore")

In [14]:
y_test = pd.read_csv("../data/processed/target_validate.csv")
X_test = pd.read_csv("../data/processed/features_validate.csv")
features=X_test.columns.tolist()

In [15]:
filename = '../models/best_model.pkl'
model = pickle.load(open(filename, 'rb'))

In [16]:
# Usando el modelo para predecir

X_test['probability']  = model.predict_proba(X_test[features])[:,1]
X_test['prediction']  = model.predict(X_test[features])

In [17]:
#Resumen de todas las métricas del modelo
metricsRfc = pd.DataFrame({'metric':['AUC','Gini','Accuracy','Precision','Recall','F1-score'],
                                'nn_test':[roc_auc_score(y_test, X_test.probability),
                                        (roc_auc_score(y_test, X_test.probability)*2-1),
                                        accuracy_score(y_test, X_test.prediction),
                                        precision_score(y_test, X_test.prediction, pos_label='satisfied'),
                                        recall_score(y_test, X_test.prediction, pos_label='satisfied'),
                                        f1_score(y_test, X_test.prediction, pos_label='satisfied')]})

In [18]:
print(confusion_matrix(y_test, X_test.prediction))
print(metricsRfc)

[[2200   66]
 [  91 1643]]
      metric   nn_test
0        AUC  0.993527
1       Gini  0.987055
2   Accuracy  0.960750
3  Precision  0.961381
4     Recall  0.947520
5   F1-score  0.954400


### Script 5: Preparación de Datos de Score (Automatización)

In [19]:
import numpy as np 
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [61]:
df = pd.read_csv("../data/raw/data_score.csv")
columns_all = pd.read_csv("../data/processed/columns_train.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25976 entries, 0 to 25975
Data columns (total 25 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         25976 non-null  int64  
 1   id                                 25976 non-null  int64  
 2   Gender                             25976 non-null  object 
 3   Customer Type                      25976 non-null  object 
 4   Age                                25976 non-null  int64  
 5   Type of Travel                     25976 non-null  object 
 6   Class                              25976 non-null  object 
 7   Flight Distance                    25976 non-null  int64  
 8   Inflight wifi service              25976 non-null  int64  
 9   Departure/Arrival time convenient  25976 non-null  int64  
 10  Ease of Online booking             25976 non-null  int64  
 11  Gate location                      25976 non-null  int

In [62]:
# Eliminar las primeras dos columnas con data innecesaria
df.drop(df.iloc[:, [0,1]], axis=1, inplace=True)

#  Columnas con valores categoricos pueden ser del tipo 'object' o 'int64'
categorical_indexes = [0, 1, 3, 4] + list(range(6,20))
print(categorical_indexes)
df.iloc[:, categorical_indexes] = df.iloc[:, categorical_indexes].astype('category')

# Como el porcentaje de los calores vacios es menor al 1% podriamos eliminar o reemplaxar por la mediana estos valores
df = df.dropna()
print(df.info())

[0, 1, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
<class 'pandas.core.frame.DataFrame'>
Index: 25893 entries, 0 to 25975
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   Gender                             25893 non-null  object  
 1   Customer Type                      25893 non-null  object  
 2   Age                                25893 non-null  int64   
 3   Type of Travel                     25893 non-null  object  
 4   Class                              25893 non-null  object  
 5   Flight Distance                    25893 non-null  int64   
 6   Inflight wifi service              25893 non-null  category
 7   Departure/Arrival time convenient  25893 non-null  category
 8   Ease of Online booking             25893 non-null  category
 9   Gate location                      25893 non-null  category
 10  Food and drink                     25893 non-n

In [42]:
# Dividir la data en numerica y categorica
numerical_columns = [c for c in df.columns if df[c].dtype.name != 'category']
numerical_columns.remove('satisfaction')
categorical_columns = [c for c in df.columns if df[c].dtype.name == 'category']
df_describe = df.describe(include = ['category'])

# Divir las columnas categoricas en binarias y no binarias
binary_columns = [c for c in categorical_columns if df_describe[c]['unique'] == 2]
nonbinary_columns = [c for c in categorical_columns if df_describe[c]['unique'] > 2]

In [45]:
# Codificación
original_df = df[binary_columns].copy()
df[binary_columns] = df[binary_columns].astype('category').apply(lambda x: x.cat.codes)
df[binary_columns] = df[binary_columns].astype('category')

# Tabla de equivalencia
list_transformation = []
list_transformation = [(col, val, code) for col in binary_columns
                                        for val, code in zip(original_df[col].unique(), df[col].unique())]

df_transform = pd.DataFrame(list_transformation, columns=["Nombre columna", "Valor Original", "Categoria"])
df_transform

,Nombre columna,Valor Original,Categoria


In [38]:
df_nonbinary = pd.get_dummies(df[nonbinary_columns])
print(df_nonbinary.columns.tolist())
df_numerical = df[numerical_columns]
print(df_numerical.columns.tolist())



['Inflight wifi service_0', 'Inflight wifi service_1', 'Inflight wifi service_2', 'Inflight wifi service_3', 'Inflight wifi service_4', 'Inflight wifi service_5', 'Departure/Arrival time convenient_0', 'Departure/Arrival time convenient_1', 'Departure/Arrival time convenient_2', 'Departure/Arrival time convenient_3', 'Departure/Arrival time convenient_4', 'Departure/Arrival time convenient_5', 'Ease of Online booking_0', 'Ease of Online booking_1', 'Ease of Online booking_2', 'Ease of Online booking_3', 'Ease of Online booking_4', 'Ease of Online booking_5', 'Gate location_1', 'Gate location_2', 'Gate location_3', 'Gate location_4', 'Gate location_5', 'Food and drink_0', 'Food and drink_1', 'Food and drink_2', 'Food and drink_3', 'Food and drink_4', 'Food and drink_5', 'Online boarding_0', 'Online boarding_1', 'Online boarding_2', 'Online boarding_3', 'Online boarding_4', 'Online boarding_5', 'Seat comfort_1', 'Seat comfort_2', 'Seat comfort_3', 'Seat comfort_4', 'Seat comfort_5', 'Inf

In [44]:
df_nonbinary = pd.get_dummies(df[nonbinary_columns])
df_numerical = df[numerical_columns]
df_numerical = (df_numerical - df_numerical.mean(axis = 0))/df_numerical.std(axis = 0)
df_final = pd.concat((df_numerical, df_nonbinary, df[binary_columns], df['satisfaction']), axis = 1)

TypeError: Could not convert ['Business travelBusiness travelBusiness travelBusiness travelBusiness travelBusiness travelBusiness travelBusiness travelBusiness travelBusiness travelBusiness travelBusiness travelBusiness travelBusiness travelBusiness travelPersonal TravelBusiness travelPersonal TravelPersonal TravelBusiness travelBusiness travelPersonal TravelPersonal TravelBusiness travelBusiness travelPersonal TravelBusiness travelBusiness travelBusiness travelBusiness travelPersonal TravelBusiness travelBusiness travelPersonal TravelBusiness travelPersonal TravelBusiness travelBusiness travelBusiness travelPersonal TravelBusiness travelBusiness travelBusiness travelBusiness travelBusiness travelBusiness travelBusiness travelPersonal TravelBusiness travelPersonal TravelPersonal TravelBusiness travelBusiness travelBusiness travelBusiness travelBusiness travelBusiness travelPersonal TravelPersonal TravelBusiness travelBusiness travelPersonal TravelPersonal TravelBusiness travelBusiness 

In [31]:
columns_now = df_final.columns.to_list()
for column in columns_all["Columns"]:
    if column not in columns_now:
       df_final[column] = 0
    
columns = columns_all["Columns"].to_list()
df_final = df_final[columns]
df_final = df_final.drop(['satisfaction'],axis=1)

NameError: name 'df_final' is not defined

In [64]:
df_final.to_csv("../data/processed/data_score.csv", index=False)

### Scipt 6: Código de Scoring (Automatización)

In [63]:
import numpy as np
import pandas as pd
import pickle
import warnings

warnings.filterwarnings("ignore")

In [65]:
df = pd.read_csv("../data/processed/data_score.csv")
columns_all = pd.read_csv("../data/processed/columns_train.csv")

In [66]:
filename = '../models/best_model.pkl'
model = pickle.load(open(filename, 'rb'))

In [67]:
scores=model.predict(df).reshape(-1,1)

In [68]:
# Exportamos el resultado del modelo para cargarlo en el Feature Store o Data Mart de Modelos
# Le asignamos nombres a las columnas
df_score = pd.DataFrame(scores, columns=['PREDICT'])
# Exportamos la solucion
df_score.to_csv('../data/scores/final_score.csv')